In [1]:
import os
import pandas as pd
import numpy as np
import re


In [2]:
data = pd.read_csv("enron_emailsdf.csv")
data.head(5)


,Unnamed: 0,Message-ID,Date,From,To,Subject,Path,Origin,FileName,Content,Content Length,Reply,Jail
0,0,<29790972.1075855665306.JavaMail.evans@thyme>,2000-12-13 18:41:00-08:00,1.11913372.-2@multexinvestornetwork.com,pallen@enron.com,"December 14, 2000 - Bear Stearns' predictions ...",\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,In today's Daily Update you'll find free repor...,8486,0,0
1,1,<21975671.1075855665520.JavaMail.evans@thyme>,2000-12-13 08:35:00-08:00,messenger@ecm.bloomberg.com,NaN,Bloomberg Power Lines Report,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,Here is today's copy of Bloomberg Power Lines....,16252,0,0
2,2,<7452188.1075855667684.JavaMail.evans@thyme>,2000-10-09 07:16:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
3,3,<23790115.1075855667708.JavaMail.evans@thyme>,2000-10-09 07:00:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
4,4,<5860470.1075855667730.JavaMail.evans@thyme>,2000-10-05 06:26:00-07:00,phillip.allen@enron.com,david.delainey@enron.com,NaN,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,"Dave, \n\n Here are the names of the west desk...",120,0,0


In [3]:
data.shape

(224107, 13)

In [4]:
# number of unique individuals
data['Origin'].nunique()

258

In [5]:
clean_data = data.copy()

In [6]:
# data containing known sender and recipient addresses for social mapping
clean_data = clean_data.dropna(subset=['From','To'])
clean_data.shape

(217170, 13)

In [7]:
# checking to see if any employee names were dropped when dropping null email addresses
clean_data['Origin'].nunique()

258

In [8]:
# emails sent and recieved internally
to_enron = clean_data[clean_data['To'].str.contains('enron.com')]
enron = to_enron[to_enron['From'].str.contains('enron.com')]

# emails sent to external addresses
to_outside = clean_data[~clean_data['To'].str.contains('enron.com')]

In [9]:
enron.head()

,Unnamed: 0,Message-ID,Date,From,To,Subject,Path,Origin,FileName,Content,Content Length,Reply,Jail
2,2,<7452188.1075855667684.JavaMail.evans@thyme>,2000-10-09 07:16:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
3,3,<23790115.1075855667708.JavaMail.evans@thyme>,2000-10-09 07:00:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
4,4,<5860470.1075855667730.JavaMail.evans@thyme>,2000-10-05 06:26:00-07:00,phillip.allen@enron.com,david.delainey@enron.com,NaN,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,"Dave, \n\n Here are the names of the west desk...",120,0,0
5,5,<14670081.1075855667751.JavaMail.evans@thyme>,2000-10-05 05:55:00-07:00,phillip.allen@enron.com,paula.harris@enron.com,Re: 2001 Margin Plan,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,"Paula,\n\n 35 million is fine\n\nPhillip",36,1,0
6,6,<2101427.1075855667773.JavaMail.evans@thyme>,2000-10-04 09:23:00-07:00,phillip.allen@enron.com,ina.rangel@enron.com,"Var, Reporting and Resources Meeting",\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,969,0,0


In [10]:
enron.to_csv('temp/internal_emails.csv')

In [11]:
to_outside.head(5)

,Unnamed: 0,Message-ID,Date,From,To,Subject,Path,Origin,FileName,Content,Content Length,Reply,Jail
7,7,<19431120.1075855667795.JavaMail.evans@thyme>,2000-10-03 09:30:00-07:00,phillip.allen@enron.com,pallen70@hotmail.com,Westgate,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,1721,0,0
9,9,<31503589.1075855667838.JavaMail.evans@thyme>,2000-10-03 09:13:00-07:00,phillip.allen@enron.com,bs_stone@yahoo.com,NaN,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,"Brenda,\n\nPlease use the second check as the ...",141,0,0
10,10,<25449858.1075855667859.JavaMail.evans@thyme>,2000-10-03 03:17:00-07:00,phillip.allen@enron.com,stouchstone@natsource.com,Re: Not business related..,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,I think Fletch has a good CPA. I am still doi...,58,1,0
11,11,<17435182.1075855667880.JavaMail.evans@thyme>,2000-10-02 02:19:00-07:00,phillip.allen@enron.com,bs_stone@yahoo.com,Re: Original Sept check/closing,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,"Brenda,\n\n Please use the second check as my ...",144,1,0
12,12,<11483563.1075855665726.JavaMail.evans@thyme>,2000-12-13 06:04:00-08:00,aod@newsdata.com,western.price.survey.contacts@ren-6.cais.net,Special report coming from NewsData,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,Our Sacramento correspondent just exited a new...,310,0,0


In [12]:
to_outside.to_csv("temp/to_external.csv")

In [13]:
# Arther
accounting = ["arthur anderson", "Arthur Andersen", "arthuranderson", "Nancy","Nancy Temple", "David Duncan", "Duncan","david duncan", "accountant"]
aa_id = []
aa_corpus = []
to_ls = []
recipient_ls = []
poi_ls = []

for a in accounting:
    for body, id, to, rec, poi in zip(data['Content'], data['Message-ID'], data['To'], data['From'], data['Origin']):
        
        corpus = str(body)
        
        x=re.search(a, corpus)
        
        if x:
            aa_id.append(id)
            to_ls.append(to)
            aa_corpus.append(corpus)
            recipient_ls.append(rec)
            poi_ls.append(poi)
            
        else:
            pass
            

a_anderson_df = pd.DataFrame(list(zip(aa_id, poi_ls, recipient_ls, to_ls, aa_corpus)), columns= ["id","poi", "from", "to", "content"])

# light filtering for spam/daily newsletters etc.
filtered_anderson_df = a_anderson_df[~a_anderson_df['content'].str.contains('Copyright')]



In [14]:
filtered_anderson_df.to_csv("temp/arthur_anderson_filtered.csv")

In [15]:
# not entirely necessary but gives me peace of mind
print(len(aa_id))
print(len(poi_ls))
print(len(recipient_ls))
print(len(to_ls))
print(len(aa_corpus))


3887
3887
3887
3887
3887
